In [1]:
import numpy as np
import h5py
import torch
import torch.utils.data as data_utils
from torch.utils.data.dataset import random_split
import numpy as np
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim

/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


# Helper Clases / Functions

In [2]:
def Load_Data(num):
    if (num == -1): # All data
        X_all = []
        y_all = []
        for i in range(8):
            file_path = './../project_datasets/A0' + str(i+1) + 'T_slice.mat'
            data = h5py.File(file_path, 'r')
            X = np.copy(data['image'])
            y = np.copy(data['type'])
            X = X[:, 0:23, :]
            X_all.append(X)
            y = y[0,0:X.shape[0]:1]
            y_all.append(y)
        A, N, E, T = np.shape(X_all)
        X_all = np.reshape(X_all, (A*N, E, T))
        y_all = np.reshape(y_all, (-1))
        y_all = y_all - 769
        ## Remove NAN
        index_Nan = []
        for i in range(A*N):
            for j in range(E):
                if (any(np.isnan(X_all[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X_all = np.delete(X_all, index_Nan, axis=0)
        y_all = np.delete(y_all, index_Nan)
        return (X_all, y_all)
    else:
        file_path = './../project_datasets/A0' + str(num) + 'T_slice.mat'
        data = h5py.File(file_path, 'r')
        X = np.copy(data['image'])
        y = np.copy(data['type'])
        X = X[:, 0:23, :]
        y = y[0,0:X.shape[0]:1]
        y = y - 769
         ## Remove NAN
        N, E, T = np.shape(X)
        index_Nan = []
        for i in range(N):
            for j in range(E):
                if (any(np.isnan(X[i,j])) == True):
                    index_Nan.append(i)
        index_Nan = list(set(index_Nan))
        X = np.delete(X, index_Nan, axis=0)
        y = np.delete(y, index_Nan)
        return (X, y)

# Load Data

In [3]:
X, y = Load_Data(-1) # -1 to load all datas
N, E, T = np.shape(X)
print (np.shape(X))

(2280, 23, 1000)


# Make DataLoaders

In [14]:
bs_train = 200
bs_val = 100
bs_test = 100
data = data_utils.TensorDataset(torch.Tensor(X[:,:,0:100]), torch.Tensor(y))
dset = {}
dataloaders = {}
dset['train'], dset['val'], dset['test'] = random_split(data, [N-bs_val-bs_test, bs_val, bs_test])
dataloaders['train'] = data_utils.DataLoader(dset['train'], batch_size=bs_train, shuffle=True, num_workers=1)
dataloaders['val'] = data_utils.DataLoader(dset['val'], batch_size=bs_val, shuffle=True, num_workers=1)
dataloaders['test'] = data_utils.DataLoader(dset['test'], batch_size=bs_test, shuffle=True, num_workers=1)

# Define Model Class

In [1]:
class myChronoNet(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myChronoNet, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        # First Inception layer
        self.conv11 = nn.Conv1d(23, 32, 2, stride=2)
        self.conv12 = nn.Conv1d(23, 32, 4, stride=2, padding=1)
        self.conv13 = nn.Conv1d(23, 32, 8, stride=2, padding=3)
        # Second Inception layer
        self.conv21 = nn.Conv1d(96, 32, 2, stride=2)
        self.conv22 = nn.Conv1d(96, 32, 4, stride=2, padding=1)
        self.conv23 = nn.Conv1d(96, 32, 8, stride=2, padding=3)
        # Third Inception layer
        self.conv31 = nn.Conv1d(96, 32, 2, stride=2)
        self.conv32 = nn.Conv1d(96, 32, 4, stride=2, padding=1)
        self.conv33 = nn.Conv1d(96, 32, 8, stride=2, padding=3)
        #self.conv_13 = nn.Conv2d()
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.gru1 = nn.GRU(32*3, hidden_dim, num_layer, dropout=0.5)
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, num_layer, dropout=0.5)
        self.gru3 = nn.GRU(hidden_dim*2, hidden_dim, num_layer, dropout=0.5)
        self.gru4 = nn.GRU(hidden_dim*3, hidden_dim, num_layer, dropout=0.5)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        out_conv11 = self.conv11(x)
        out_conv12 = self.conv12(x)
        out_conv13 = self.conv13(x)
        out_conv1 = torch.cat((out_conv11, out_conv12, out_conv13), 1)
        out_conv21 = self.conv21(out_conv1)
        out_conv22 = self.conv22(out_conv1)
        out_conv23 = self.conv23(out_conv1)
        out_conv2 = torch.cat((out_conv21, out_conv22, out_conv23), 1)
        out_conv31 = self.conv31(out_conv2)
        out_conv32 = self.conv32(out_conv2)
        out_conv33 = self.conv33(out_conv2)
        out_conv3 = torch.cat((out_conv31, out_conv32, out_conv33), 1)
        # N, C, L --> L, N, C
        out_conv3 = out_conv3.permute(2,0,1)
        out_gru1, _ = self.gru1(out_conv3)
        out_gru2, _ = self.gru2(out_gru1)
        out_gru12 = torch.cat((out_gru1, out_gru2), 2)
        out_gru3, _ = self.gru3(out_gru12)
        out_gru321 = torch.cat((out_gru1, out_gru2, out_gru3), 2)
        out_gru4, _ = self.gru4(out_gru321)
        out_gru4 = out_gru4[-1, :, :] # taking the last time seq
        out = self.classifier(out_gru4)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

class myGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myGRU, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm1 = nn.GRU(input_dim, hidden_dim, num_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x = x.transpose(1,2)
        out_gru1, _ = self.gru1(x)
        out_gru1 = out_gru1[:,-1,:]
        out_lin = self.classifier1(out_gru1)
        out = self.classifier(out_lin)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label
class myCONVGRU(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myGRU, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.conv_temp = nn.Conv2d(1,40,tuple([1,25]))
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.gru1 = nn.GRU(input_dim, hidden_dim, num_layer)
        self.act1 = nn.Tanh()
        self.gru2 = nn.GRU(hidden_dim, hidden_dim, num_layer)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x = x.permute(2,0,1)
        out_gru1, _ = self.gru1(x)
        out_act1 = self.act1(out_gru1)
        out_gru2, _ = self.gru2(out_act1)
        out_gru2 = out_gru2[-1, :, :] # taking the last time seq
        out = self.classifier(out_gru2)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label
class myLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x = x.transpose(1,2)
        out_lstm1, _ = self.lstm1(x)
        out_lstm1 = out_lstm1[:,-1,:]
        out_lin = self.classifier1(out_lstm1)
        out = self.classifier(out_lin)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label
    
class myLSTMDO(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myLSTMDO, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.lstm1 = nn.LSTM(input_dim, hidden_dim, num_layer, batch_first=True, dropout=0.5)
        self.lstm2 = nn.LSTM(hidden_dim, hidden_dim, num_layer, batch_first=True, dropout=0.5)
        self.lstm3 = nn.LSTM(hidden_dim, hidden_dim, num_layer, batch_first=True, dropout=0.5)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x = x.transpose(1,2)
        out_lstm1, _ = self.lstm1(x)
        out_lstm2, _ = self.lstm2(out_lstm1)
        out_lstm3, _ = self.lstm3(out_lstm2)
        out_lstm3 = out_lstm3[:,-1,:]
        out_lin = self.classifier(out_lstm3)
        out = self.classifier2(out_lin)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            # Flip axis first
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

class myCONVLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layer, num_class):
        super(myCONVLSTM, self).__init__()
        self.num_layer = num_layer
        self.hidden_dim = hidden_dim
        self.conv_temp = nn.Conv2d(1,40,tuple([1,25]))
        self.conv_elec = nn.Conv3d(1,23,tuple([40, 23, 1]))
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layer, dropout=0.5)
        self.classifier = nn.Linear(hidden_dim, num_class)
    def forward(self, x):
        x.unsqueeze_(1)
        out_conv_temp = self.conv_temp(x)
        out_conv_temp = out_conv_temp.unsqueeze_(1)
        out_conv_elec = self.conv_elec(out_conv_temp)
        out_conv_elec_sque= torch.squeeze(out_conv_elec)
        out_swap = out_conv_elec_sque.permute(2,0,1)
        out_lstm, _ = self.lstm(out_swap)
        out_lstm = out_lstm[-1, :, :]
        out = self.classifier(out_lstm)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

class mySHALLOWCONV(nn.Module):
    def __init__(self, num_class):
        super(mySHALLOWCONV, self).__init__()
        self.conv_temp = nn.Conv2d(1,40,tuple([1,25]))
        self.conv_elec = nn.Conv3d(1,40,tuple([40, 23, 1]))
        self.pool = nn.AvgPool2d(tuple([1,47]))
        self.classifier = nn.Linear(40*20, num_class)
    def forward(self, x):
        N, H, W = x.size()
        x.unsqueeze_(1)
        out_conv_temp = self.conv_temp(x)
        out_conv_temp = out_conv_temp.unsqueeze_(1)
        out_conv_elec = self.conv_elec(out_conv_temp)
        out_conv_elec = torch.squeeze(out_conv_elec) # shape: [N, 40, 976]
        out_conv_elec.unsqueeze_(1)
        out_pool = self.pool(out_conv_elec) 
        out_pool = torch.squeeze(out_pool) # shape: [N, 40, 20]
        out_pool = out_pool.view(N, -1) # shape: [N, 800]
        out = self.classifier(out_pool)
        return out
    def check_accuracy(self, dataloader):
        total_correct = 0
        total_label = 0
        for i_batch, sample_batched in enumerate(dataloader):
            X_sample, y_sample = sample_batched
            X_sample, y_sample = Variable(X_sample), Variable(y_sample)
            out = self.forward(X_sample.cuda())
            _, pred = torch.max(out, 1)
            num_correct = np.sum(pred.data.cpu().numpy() == y_sample.data.cpu().numpy())
            total_correct += num_correct
            total_label += len(pred)
        return  total_correct / total_label

NameError: name 'nn' is not defined

# Define Model

In [34]:
dtype = torch.cuda.FloatTensor
hidden_dim = 32
num_classes = 4
num_epoches = 10
#model = myGRU(E, hidden_dim, 1, num_classes)
#model = myCONVGRU(E, hidden_dim, 1, num_classes)
#model = myLSTM(E, hidden_dim, 1, num_classes)
#model = myLSTMDO(E, hidden_dim, 1, num_classes)
#model = myCONVLSTM(E, hidden_dim, 1, num_classes)
#model = myChronoNet(E, hidden_dim, 1, num_classes)
#model = mySHALLOWCONV(4)
model.type(dtype)
loss_fn = nn.CrossEntropyLoss().type(dtype)
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# Training

In [35]:
best_acc = 0

In [39]:
for epoch in range(num_epoches):
    for i, data in enumerate(dataloaders['train'], 0):
        X_train, y_train = data
        # Wrap them in Variable
        X_train, y_train = Variable(X_train), Variable(y_train)
        # forward + backward + optimize
        out = model(X_train.cuda())
        # print (out)
        loss = loss_fn(out, y_train.long().cuda())
        print('(%d batch) loss: %f' % (i, loss))
        # zero the parameter gradients
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    train_acc = model.check_accuracy(dataloaders['train'])
    val_acc = model.check_accuracy(dataloaders['val'])
    print('(Epoch %d / %d) train_acc: %f; val_acc: %f' % (epoch+1, num_epoches, train_acc, val_acc))
    if (val_acc > best_acc):
        best_acc = val_acc
        torch.save(model, 'best_GRU.pt')
print (best_acc)

(0 batch) loss: 0.924785
(1 batch) loss: 0.841989
(2 batch) loss: 0.892003
(3 batch) loss: 0.826646
(4 batch) loss: 0.986290
(5 batch) loss: 0.885639
(6 batch) loss: 0.939097
(7 batch) loss: 0.911761
(8 batch) loss: 0.946886
(9 batch) loss: 0.923872
(10 batch) loss: 0.790424
(Epoch 1 / 10) train_acc: 0.645192; val_acc: 0.510000
(0 batch) loss: 0.881480
(1 batch) loss: 0.991666
(2 batch) loss: 0.938316
(3 batch) loss: 0.865532
(4 batch) loss: 0.918177
(5 batch) loss: 0.917049
(6 batch) loss: 0.807217
(7 batch) loss: 0.893480
(8 batch) loss: 0.861838
(9 batch) loss: 0.860324
(10 batch) loss: 0.812225
(Epoch 2 / 10) train_acc: 0.659615; val_acc: 0.500000
(0 batch) loss: 0.807760
(1 batch) loss: 0.806120
(2 batch) loss: 0.905030
(3 batch) loss: 0.919659
(4 batch) loss: 0.827668
(5 batch) loss: 0.942985
(6 batch) loss: 0.929319
(7 batch) loss: 0.849261
(8 batch) loss: 0.926692
(9 batch) loss: 0.905181
(10 batch) loss: 0.792160
(Epoch 3 / 10) train_acc: 0.662500; val_acc: 0.460000
(0 batch) 

# Best Model Test

In [42]:
best_model = torch.load('best_GRU.pt')
print (best_model.check_accuracy(dataloaders['test']))

0.34


/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/ipykernel_launcher.py:75: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
/home/andrew/Desktop/Project/model_explore/.env/lib/python3.5/site-packages/ipykernel_launcher.py:77: UserWarning: RNN module weights are not part of single contiguous chunk of memory. This means they need to be compacted at every call, possibly greatly increasing memory usage. To compact weights again call flatten_parameters().
